In [ ]:
!pip install mlflow
!pip install onnx
!pip install tf2onnx

In [ ]:
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import mlflow
import os
import cv2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print('Training data shape: ', train_images.shape, train_labels.shape)
print('Testing data shape: ', test_images.shape, test_labels.shape)
classes = np.unique(train_labels)
nClasses = len(classes)
print('Total number of outputs: ', nClasses)
print('Output classes: ', classes)
plt.figure(figsize=[4, 2])
plt.subplot(121)
plt.imshow(train_images[0])
plt.title('Ground Truth: {}'.format(train_labels[0]))
plt.subplot(122)
plt.imshow(test_images[0])
plt.title('Ground Truth: {}'.format(test_labels[0]))
plt.show()

In [ ]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
print('Original label: ', train_labels[0])
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print('After conversion to categorical (one-hot): ', train_labels[0])

In [ ]:
batch_size = 256
epochs = 25
optimizer = RMSprop()
loss_function = 'categorical_crossentropy'
metrics = ['accuracy']

In [ ]:
mlflow.set_tracking_uri("http://52.146.88.221:5000")
mlflow.set_experiment("CIFAR10-Image-Classification")
with mlflow.start_run(run_name="CNN_CIFAR10"):
    mlflow.log_param("optimizer", "RMSprop")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("loss_function", loss_function)
    model = Sequential([
        Input(shape=train_images.shape[1:]),
        Conv2D(32, (3, 3), padding='same', activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(nClasses, activation='softmax')
    ])
    model.summary()
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)
    history = model.fit(
        train_images, train_labels,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_split=0.20
    )
    plt.figure(figsize=[8, 6])
    plt.plot(history.history['loss'], 'r', linewidth=3.0)
    plt.plot(history.history['val_loss'], 'b', linewidth=3.0)
    plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.title('Loss Curves', fontsize=16)
    plt.show()
    plt.savefig("Loss-Curves.png")
    plt.figure(figsize=[8, 6])
    plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
    plt.plot(history.history['val_accuracy'], 'b', linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.title('Accuracy Curves', fontsize=16)
    plt.show()
    plt.savefig("Accuracy-Curves.png")
    mlflow.log_artifact("Loss-Curves.png")
    mlflow.log_artifact("Accuracy-Curves.png")
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_accuracy", test_accuracy)

In [ ]:
labels =  ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plt.figure(figsize=(13, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.axis('off')
    plt.imshow(test_images[i])
    gt_string = f'Ground Truth: {labels[np.argmax(test_labels[i])]}'
    pred_string = f'Prediction: {labels[np.argmax(model.predict(test_images[i:i+1]))]}'
    plt.title(f'{gt_string}\n{pred_string}')
    plt.tight_layout()
plt.show()

In [ ]:
model.export("models/cifar_classifier.onnx", format="onnx")

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
label_names.sort()
model_path = os.path.join('models', 'cifar_classifier.onnx')
net = cv2.dnn.readNetFromONNX(model_path)
img_original = cv2.imread('image_2.png')
img = img_original.copy()
img = cv2.resize(img,(32,32))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array([img]).astype('float64') / 255.0
net.setInput(img)
out = net.forward()
index = np.argmax(out[0])
prob = np.max(out[0])
label =  label_names[index]
text = 'Predicted: {}, Probability {:.2f}%'.format(label, prob*100)
cv2.putText(img_original, text, (5, 40),  cv2.FONT_HERSHEY_COMPLEX, 1, (100, 20, 255), 2)
plt.figure(figsize=(20,10))
plt.imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()